In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os , re
import numpy as np
import pandas as pd
import os , re
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import spacy
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize

In [13]:
train_txt = []
test_txt = []
for line in open(r"movie_data/full_train.txt",'r',encoding='utf-8'):
    train_txt.append(line.strip())
for line in open(r"movie_data/full_test.txt",'r',encoding='utf-8'):
    test_txt.append(line.strip())
target = [1 if i < 12500 else 0 for i in range(25000)]

In [14]:
print(train_txt[5])

This isn't the comedic Robin Williams, nor is it the quirky/insane Robin Williams of recent thriller fame. This is a hybrid of the classic drama without over-dramatization, mixed with Robin's new love of the thriller. But this isn't a thriller, per se. This is more a mystery/suspense vehicle through which Williams attempts to locate a sick boy and his keeper.<br /><br />Also starring Sandra Oh and Rory Culkin, this Suspense Drama plays pretty much like a news report, until William's character gets close to achieving his goal.<br /><br />I must say that I was highly entertained, though this movie fails to teach, guide, inspect, or amuse. It felt more like I was watching a guy (Williams), as he was actually performing the actions, from a third person perspective. In other words, it felt real, and I was able to subscribe to the premise of the story.<br /><br />All in all, it's worth a watch, though it's definitely not Friday/Saturday night fare.<br /><br />It rates a 7.7/10 from...<br /><

In [15]:
nlp = spacy.load("en_core_web_lg")
sw = stopwords.words("english")

In [18]:
no_space = re.compile("(\")|(\.)|(\d+)|(\:)|(\;)|(\()|(\))|(\!)|(\?)|(\,)|(\[)|(\])")
space = re.compile('(\')|(<br\s*/><br\s*/>)|(\-)|(\/)')
rep1 = ""
rep2 = " "
    
def process(txt):
    txt = [no_space.sub(rep1,line.lower()) for line in txt]
    txt = [space.sub(rep2,line.lower()) for line in txt]
    return txt

In [20]:
txt1 = process(train_txt)
txt2 = process(test_txt)

In [29]:
def clean(txt):
    cleaned_doc = []
    for line in txt:
        cleaned_txt = ""
        for tok in word_tokenize(line):
            if tok not in sw:
                cleaned_txt += tok + ' '
        cleaned_doc.append(cleaned_txt.strip())
    return cleaned_doc

In [31]:
txt1 = clean(txt1)
txt2 = clean(txt2)

In [32]:
cv = CountVectorizer()
train = cv.fit_transform(txt1)
test = cv.transform(txt2)
x_train,x_test,y_train,y_test = train_test_split(train,target,test_size=0.15)

In [34]:
test.shape

(25000, 86301)

In [36]:
lr = LogisticRegression(C=0.05)
lr.fit(x_train, y_train)
print ("Training Accuracy: %s" % (accuracy_score(y_train, lr.predict(x_train))))

Training Accuracy: 0.9623529411764706


In [37]:
print ("Validation Accuracy: %s" % (accuracy_score(y_test, lr.predict(x_test))))

Validation Accuracy: 0.8826666666666667


In [38]:
print ("Test Accuracy: %s" % (accuracy_score(target, lr.predict(test))))

Test Accuracy: 0.87936


In [40]:
len(cv.get_feature_names_out())

86301

In [41]:
lr.coef_

array([[-0.00308606,  0.04345038,  0.04464074, ...,  0.00030823,
        -0.01485266, -0.01611065]])

In [44]:
len(lr.coef_[0])

86301

In [46]:
feat_coef = {word: coef for word, coef in zip(cv.get_feature_names_out(),lr.coef_[0])}
list(feat_coef.items())[:10]

[('____', -0.003086058167396207),
 ('_____', 0.043450381258337153),
 ('______', 0.0446407367864826),
 ('_________', 0.011430557587571114),
 ('____________________________________', 0.0013114000206186347),
 ('_____________________________________', 0.0015367651154646622),
 ('__________________________________________________________________',
  0.0041943313464517205),
 ('____insert', -0.0002021615939818414),
 ('_a', -0.0017948613295984712),
 ('_absolute', -5.665914085335977e-05)]

In [47]:
for p in sorted(feat_coef.items(),key=lambda x: x[1],reverse=True)[:5]:
    print(p)

('excellent', 0.773902260434357)
('perfect', 0.6765370620895124)
('wonderful', 0.6161483008393006)
('favorite', 0.5767015121584536)
('amazing', 0.5644034225852854)


In [48]:
for p in sorted(feat_coef.items(),key=lambda x: x[1])[:5]:
    print(p)

('worst', -1.2138533214898892)
('waste', -1.1859916272886992)
('awful', -0.9567724912759495)
('disappointment', -0.7907383424131592)
('boring', -0.7599679158459768)
